In [1]:
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [2]:
df = pd.read_csv('~/Downloads/amazon_co-ecommerce_sample.csv')

In [3]:
df

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,44d6967f083825a5de36ad4865a65bcd,Batman 1966 TV Series Action Figures - The Rid...,Mattel,£22.95,5 new,3,3.0,5.0 out of 5 stars,Hobbies > Collectible Figures & Memorabilia > ...,http://www.amazon.co.uk/Batman-1966-Series-Act...,DC 66 Batman Classic TV Series 6 Inch Riddler ...,Technical Details Item Weight136 g Product Dim...,DC 66 Batman Classic TV Series 6 Inch Riddler ...,NaN,NaN,Realistic // 5.0 // 31 Mar. 2014 // By\n \n...,"{""seller""=>[{""Seller_name_1""=>""Star Action Fig..."
9996,08f0747b6fc6687215ffb994c3a6fb32,"Star Wars Costume, Kids Stormtrooper Costume S...",Star Wars,£39.99,NaN,1,3.0,4.0 out of 5 stars,Characters & Brands > Star Wars > Toys,http://www.amazon.co.uk/Storm-Trooper-Costume-...,NaN,Technical Details Additional Information AS...,NaN,http://www.amazon.co.uk/Storm-Trooper-Costume-...,NaN,... what I see my grandson us going to have fu...,NaN
9997,bf6cc073f8f24e6e338190fa16f6ee9d,Defiance Lawkeeper Metal Badge Prop R

In [4]:
df.shape

(10000, 17)

In [5]:
df['customer_reviews'][3]

"I love it // 5.0 // 22 July 2013 // By\n    \n    Lilla Lukacs\n  \n on 22 July 2013 // I love it. Perfect with the earlier ordered locomotive.Again: I would recommend it to the masters of the topic. It's not just a toy."

In [6]:
df.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


In [7]:
#separate review column by // breaks
reviews = df['customer_reviews'].str.split("//", n=4, expand=True)

In [8]:
reviews.head()

,0,1,2,3,4
0,Worth Buying For The Pictures Alone (As Ever),4.0,6 April 2014,By\n \n Copnovelist\n \n on 6 April 2014,Part of the magic for me growing up as a boy ...
1,Four Stars,4.0,18 Dec. 2015,By\n \n kenneth bell\n \n on 18 Dec. 2...,Very happy with the communication with funkyb...
2,**Highly Recommended!**,5.0,26 May 2015,By\n \n Simon.B :-)\n \n on 26 May 2015,Simple & GREAT FUN for 5+My nephews face was ...
3,I love it,5.0,22 July 2013,By\n \n Lilla Lukacs\n \n on 22 July 2...,I love it. Perfect with the earlier ordered l...
4,Birthday present,5.0,14 April 2014,By\n \n Love my Dog\n \n on 14 April 2...,Bought this for my Grandson's birthday. He i...


In [9]:
df['review_title'] = reviews[0]
df['rating'] = reviews[1]
df['review_date'] = reviews[2]
df['customer_name'] = reviews[3]
df['review'] = reviews[4]
df.drop(columns='customer_reviews', inplace=True)

In [10]:
df.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,...,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,sellers,review_title,rating,review_date,customer_name,review
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ...",Worth Buying For The Pictures Alone (As Ever),4.0,6 April 2014,By\n \n Copnovelist\n \n on 6 April 2014,Part of the magic for me growing up as a boy ...
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ...",Four Stars,4.0,18 Dec. 2015,By\n \n kenneth bell\n \n on 18 Dec. 2...,Very happy with the communication with funkyb...
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel...",**Highly Recommended!**,5.0,26 May 2015,By\n \n Simon.B :-)\n \n on 26 May 2015,Simple & GREAT FUN for 5+My nephews face was ...
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,NaN,I love it,5.0,22 July 2013,By\n \n Lilla Lukacs\n \n on 22 July 2...,I love it. Perfect with the earlier ordered l...
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,NaN,Birthday present,5.0,14 April 2014,By\n \n Love my Dog\n \n on 14 April 2...,Bought this for my Grandson's birthday. He i...


In [11]:
def remove_punctuation(text):
    no_punct = "".join([c for c in str(text) if c not in string.punctuation])
    return no_punct

In [12]:
df['review'] = df['review'].apply(lambda x: remove_punctuation(x))

In [13]:
df['review'].head()

0     Part of the magic for me growing up as a boy ...
1     Very happy with the communication with funkyb...
2     Simple  GREAT FUN for 5My nephews face was am...
3     I love it Perfect with the earlier ordered lo...
4     Bought this for my Grandsons birthday  He is ...
Name: review, dtype: object

In [14]:
#instantiate Tokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [15]:
df['review'] = df['review'].apply(lambda x: tokenizer.tokenize(x.lower()))

In [16]:
df['review'].head(20)

0     [part, of, the, magic, for, me, growing, up, a...
1     [very, happy, with, the, communication, with, ...
2     [simple, great, fun, for, 5my, nephews, face, ...
3     [i, love, it, perfect, with, the, earlier, ord...
4     [bought, this, for, my, grandsons, birthday, h...
5     [the, victorian, style, added, to, the, charac...
6     [high, standard, model, well, worth, the, wait...
7     [this, has, gone, down, very, well, this, chri...
8     [xmas, box, for, grandsonam, sure, it, will, b...
9     [we, bought, this, and, 2, othersfor, our, gra...
10             [grandson, loved, it, hauls, good, load]
11                    [very, good, product, thank, you]
12                                         [very, good]
13    [beautiful, model, if, your, a, fan, of, the, ...
14    [great, quality, lots, of, detail, run, nic, a...
15    [fantastic, little, set, beautifully, made, an...
16    [the, product, is, as, expected, the, only, wo...
17    [my, 2, year, old, loves, chuggington, so,

In [23]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [24]:
df['review'] = df['review'].apply(lambda x : remove_stopwords(x))

In [25]:
df['review'].head(20)

0     [part, magic, growing, boy, buy, given, new, h...
1     [happy, communication, funkybuys, five, stars,...
2     [simple, great, fun, 5my, nephews, face, amazi...
3     [love, perfect, earlier, ordered, locomotiveag...
4     [bought, grandsons, birthday, currently, colle...
5     [victorian, style, added, character, husband, ...
6     [high, standard, model, well, worth, wait, rep...
7     [gone, well, christmas, adults, kids, collecti...
8                  [xmas, box, grandsonam, sure, great]
9     [bought, 2, othersfor, grandsons, love, 7, get...
10                 [grandson, loved, hauls, good, load]
11                               [good, product, thank]
12                                               [good]
13    [beautiful, model, fan, gwr, loco, must, five,...
14    [great, quality, lots, detail, run, nic, smoot...
15    [fantastic, little, set, beautifully, made, de...
16    [product, expected, worry, looks, like, connec...
17    [2, year, old, loves, chuggington, decided

In [27]:
#Instantiate lemmatizer
lemmatizer = WordNetLemmatizer()
def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

In [29]:
nltk.download('wordnet')
df['review'].apply(lambda x: word_lemmatizer(x))

[nltk_data] Downloading package wordnet to /Users/stubbs/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Exception ignored in: <function SeekableUnicodeStreamReader.__del__ at 0x120debcb0>
Traceback (most recent call last):
  File "/Users/stubbs/Documents/research/CMNS801/ass_2/venv/lib/python3.7/site-packages/nltk/data.py", line 1280, in __del__
    if not self.closed:
  File "/Users/stubbs/Documents/research/CMNS801/ass_2/venv/lib/python3.7/site-packages/nltk/data.py", line 1294, in closed
    return self.stream.closed
AttributeError: 'SeekableUnicodeStreamReader' object has no attribute 'stream'


0       [part, magic, growing, boy, buy, given, new, h...
1       [happy, communication, funkybuys, five, star, ...
2       [simple, great, fun, 5my, nephew, face, amazin...
3       [love, perfect, earlier, ordered, locomotiveag...
4       [bought, grandson, birthday, currently, collec...
                              ...                        
9995    [true, batman, classic, series, 1960s, highly,...
9996    [christmas, present, however, see, grandson, u...
9997                      [arrived, excellent, condition]
9998    [okay, hal, jordan, figure, come, go, new, one...
9999    [1st, class, kit, top, mark, bandai, good, pai...
Name: review, Length: 10000, dtype: object

In [31]:
#instantiate Stemmer
stemmer = PorterStemmer()

In [32]:
def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

In [33]:
df['review'] = df['review'].apply(lambda x: word_stemmer(x))

In [34]:
df['review']

0       part magic grow boy buy given new hornbi catal...
1       happi commun funkybuy five star 50 14 jan 2016...
2       simpl great fun 5mi nephew face amaz open birt...
3       love perfect earlier order locomotiveagain wou...
4       bought grandson birthday current collect bit t...
                              ...                        
9995    true batman classic seri 1960 highli recommend...
9996        christma present howev see grandson us go fun
9997                                   arriv excel condit
9998    okay hal jordan figur come go new one around c...
9999    1st class kit top mark bandai good paint dont ...
Name: review, Length: 10000, dtype: object